In [2]:
# predict calls with random forest


In [29]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [4]:
df = pd.read_csv('/Users/kata/Desktop/GitHub/standby_duty_forecasting/sickness_table.csv', index_col=0)
df['date'] = pd.to_datetime(df['date'])
df['year']= df['date'].dt.year
df['month']= df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week'] = df['date'].dt.isocalendar().week
df['week'] = df['week'].astype(np.int64)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1152 entries, 0 to 1151
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1152 non-null   datetime64[ns]
 1   n_sick       1152 non-null   int64         
 2   calls        1152 non-null   float64       
 3   n_duty       1152 non-null   int64         
 4   n_sby        1152 non-null   int64         
 5   sby_need     1152 non-null   float64       
 6   dafted       1152 non-null   float64       
 7   year         1152 non-null   int64         
 8   month        1152 non-null   int64         
 9   day_of_week  1152 non-null   int64         
 10  week         1152 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(7)
memory usage: 108.0 KB


In [5]:
X = df[['year', 'month', 'day_of_week',
       'week']]
y = df.calls

In [6]:
def train_test_split(data, n_test):
	return data[:n_test], data[n_test:]


x_train, x_test = train_test_split(X, 921)
y_train, y_test = train_test_split(y, 921)


In [7]:
model = XGBRegressor()

In [8]:
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [9]:
pred = model.predict(x_test)

In [10]:
pred

array([ 9072.806 ,  8155.94  ,  8627.281 ,  8043.2935,  7422.192 ,
        7884.6733,  9600.832 ,  8529.699 ,  8320.084 ,  9291.118 ,
        8657.1455,  7605.6973,  7488.711 ,  8741.474 ,  8585.317 ,
        8284.157 ,  9171.77  ,  8912.073 ,  7815.185 ,  7903.126 ,
        9934.257 ,  7836.289 ,  7670.754 , 10146.952 , 10214.008 ,
        8199.1   ,  7703.5713,  8291.624 ,  7980.8853,  7650.7715,
        7663.6313,  8252.081 ,  7476.681 ,  6912.535 ,  9193.486 ,
        8773.058 ,  7629.858 ,  8238.679 ,  7906.112 ,  6963.6895,
        6206.161 ,  7118.527 ,  8132.553 ,  8010.0664,  7530.1914,
        7514.039 ,  6927.144 ,  6106.283 ,  6801.948 ,  8014.036 ,
        6754.197 ,  8788.449 ,  8508.348 ,  6848.31  ,  6739.1255,
        7900.215 ,  8790.33  ,  8028.5747,  8247.475 ,  7871.816 ,
        7746.8223,  7057.115 ,  7358.0425,  8317.43  ,  7351.8896,
        7981.0503,  7277.7617,  7997.558 ,  6217.0977,  7158.677 ,
        6911.571 ,  7467.809 ,  9225.225 ,  8412.445 ,  7659.3

In [11]:
mean_squared_error(y_test, pred)
# 2092620.6058171096

2092620.6058171096

In [30]:
Accuracy=r2_score(y_test,pred)*100
print(" Accuracy of the model is %.2f" %Accuracy)

 Accuracy of the model is -24.77


In [12]:
# decision tree?
from sklearn.ensemble import RandomForestRegressor


In [13]:
regressor = RandomForestRegressor()

In [14]:
regressor.fit(x_train, y_train)

RandomForestRegressor()

In [15]:
y_pred = regressor.predict(x_test)

y_pred

array([9468.72, 9076.2 , 9624.78, 8909.76, 8016.42, 7651.62, 9477.24,
       9194.88, 8913.48, 9401.58, 8748.66, 7858.44, 7543.26, 9306.78,
       9108.  , 8812.26, 9256.5 , 8722.38, 7738.44, 7618.8 , 9339.6 ,
       9004.26, 8829.78, 9513.3 , 8982.78, 7929.72, 7461.42, 8505.84,
       8438.1 , 7952.58, 7915.02, 7788.48, 7429.8 , 6874.68, 8834.16,
       8736.48, 7967.64, 8203.38, 7760.88, 7082.7 , 6555.48, 7980.72,
       8411.52, 8149.68, 7884.6 , 7645.5 , 7096.5 , 6509.22, 7805.16,
       8290.92, 7618.26, 8309.52, 8013.54, 7068.36, 6823.56, 8201.7 ,
       8648.7 , 8066.1 , 8163.06, 7772.88, 7499.34, 6981.36, 7846.56,
       8124.  , 7699.38, 7914.66, 7292.34, 7538.52, 6344.34, 7826.64,
       7687.02, 7650.  , 8354.28, 7881.12, 7438.02, 6195.06, 8215.38,
       7640.94, 7602.48, 8184.96, 7967.04, 7244.94, 6269.76, 9114.66,
       7541.76, 6897.84, 6920.34, 7751.94, 7448.64, 7312.5 , 8257.62,
       8121.54, 7906.86, 7239.18, 7380.24, 6711.9 , 6841.08, 7112.58,
       7524.66, 6427

In [16]:
mean_squared_error(y_test, y_pred)
#1743722.4680571433

1773742.063994805

In [31]:
Accuracy=r2_score(y_test,y_pred)*100
print(" Accuracy of the model is %.2f" %Accuracy)

 Accuracy of the model is -3.08


In [17]:
# SVM
from sklearn.svm import SVR

In [18]:
regressor = SVR(kernel = 'rbf')
regressor.fit(X, y)

SVR()

In [19]:
y_pred = regressor.predict(x_test)


In [22]:
mean_squared_error(y_test, y_pred)
# 1970608.4805071584

1970608.4805071584

In [32]:
Accuracy=r2_score(y_test,y_pred)*100
print(" Accuracy of the model is %.2f" %Accuracy)

 Accuracy of the model is -3.08


In [23]:
# linear regression
from sklearn.linear_model import LinearRegression


In [24]:
linreg=LinearRegression()
linreg.fit(x_train,y_train)

LinearRegression()

In [25]:
y_pred=linreg.predict(x_test)


In [28]:
mean_squared_error(y_test, y_pred)
# 1728872.9009036084

1728872.9009036084

In [33]:
Accuracy=r2_score(y_test,y_pred)*100
print(" Accuracy of the model is %.2f" %Accuracy)

 Accuracy of the model is -3.08
